Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [1]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df = pd.DataFrame.from_records(ham)
df = df.append(pd.DataFrame.from_records(spam))

skipped 2248.2004-09-23.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt


C:\Users\vik\AppData\Local\Temp\ipykernel_5360\217166261.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame.from_records(spam))


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [2]:
import re

def preprocessor(e):
    # Replace non-alphabet characters with a space
    processed_string = re.sub(r'[^a-zA-Z]', ' ', e)
    # Convert the string to lowercase
    processed_string = processed_string.lower()
    return processed_string

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.

vectorizer = CountVectorizer(preprocessor=preprocessor)


# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.

X = df['content']
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.

X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.

logistic_model = LogisticRegression(max_iter=1000)  # Increase the max_iter value
logistic_model.fit(X_train_transformed, y_train)

# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.

X_test_transformed = vectorizer.transform(X_test)

# Generate predictions using the trained LogisticRegression model
y_pred = logistic_model.predict(X_test_transformed)



# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.


# Generate predictions using the trained LogisticRegression model
y_pred = logistic_model.predict(X_test_transformed)

# Calculate and print the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Calculate and print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

# Generate and print the classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))



Accuracy: 0.97
Confusion Matrix:
[[1086   20]
 [  21  424]]
Classification Report:
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98      1106
        spam       0.95      0.95      0.95       445

    accuracy                           0.97      1551
   macro avg       0.97      0.97      0.97      1551
weighted avg       0.97      0.97      0.97      1551



Step 4.

In [11]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.

feature_names = vectorizer.get_feature_names_out()

# Print the feature names
print("Feature Names:")
print(feature_names)


# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.

# Access the coefficients from the trained LogisticRegression model
coefficients = logistic_model.coef_[0]

# Print the coefficients
print("Coefficients:")
print(coefficients)



# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.

# Iterate over the coefficients and find the top 10 positive and negative features
positive_features = []
negative_features = []

# Find top positive features
top_positive_indices = sorted(range(len(coefficients)), key=lambda i: -coefficients[i])[:10]
positive_features = [feature_names[i] for i in top_positive_indices]

# Find top negative features
top_negative_indices = sorted(range(len(coefficients)), key=lambda i: coefficients[i])[:10]
negative_features = [feature_names[i] for i in top_negative_indices]

# Print the top positive and negative features
print("Top 10 Positive Features (Spam):")
print(positive_features)

print("\nTop 10 Negative Features (Ham):")
print(negative_features)



Feature Names:
['aa' 'aaa' 'aabda' ... 'zyyqywp' 'zzezrjok' 'zzo']
Coefficients:
[-2.97969801e-01  1.18376202e-04 -3.69515523e-03 ...  4.15772760e-05
  1.12806276e-02  1.10207073e-02]
Top 10 Positive Features (Spam):
['no', 'prices', 'remove', 'http', 'more', 'here', 'only', 'paliourg', 'removed', 'hello']

Top 10 Negative Features (Ham):
['attached', 'enron', 'thanks', 'daren', 'pictures', 'doc', 'neon', 'deal', 'xls', 'hpl']


Submission
1. Upload the jupyter notebook to Forage.

All Done!